# 01_data_wrangling_realized_volatility

## Key Objectives

1) Look at the different files provided by Optiver and understand the data.
* **book_train** : This file has the bid/ask price data
* **trade_train** : This file has the data on trades executed for a particular stock
* **train** : This file has the target - the realized volatility values to be predicted using book_train and trade_train data
* **test** : This file has the test data

2) Understand any missing values and impute them, if required.

3) Create and separate training and test data.


In [2]:
import pandas as pd
import numpy as np
import glob
from sb_utils import save_file

import seaborn as sns
import matplotlib.pyplot as plt

from statsmodels.tsa.stattools import acf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, learning_curve
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error



In [1]:
#pip install fastparquet

In [2]:
#pip install pyarrow

## 1.0 A peak into the 'book_train' parquet file

There is a book_train file available for each of the 112 different stocks. We shall look at this file for a few stocks to understand the data and perform any cleaning, imputing and tidying operations if required.

### Stock_id=0

In [3]:
stock_0_book_train = pd.read_parquet('../Raw_data/book_train.parquet/stock_id=0')
stock_0_book_train.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 917553 entries, 0 to 917552
Data columns (total 10 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   time_id            917553 non-null  int16  
 1   seconds_in_bucket  917553 non-null  int16  
 2   bid_price1         917553 non-null  float32
 3   ask_price1         917553 non-null  float32
 4   bid_price2         917553 non-null  float32
 5   ask_price2         917553 non-null  float32
 6   bid_size1          917553 non-null  int32  
 7   ask_size1          917553 non-null  int32  
 8   bid_size2          917553 non-null  int32  
 9   ask_size2          917553 non-null  int32  
dtypes: float32(4), int16(2), int32(4)
memory usage: 31.5 MB


In [4]:
stock_0_book_train.head(10)

,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2
0,5,0,1.001422,1.002301,1.00137,1.002353,3,226,2,100
1,5,1,1.001422,1.002301,1.00137,1.002353,3,100,2,100
2,5,5,1.001422,1.002301,1.00137,1.002405,3,100,2,100
3,5,6,1.001422,1.002301,1.00137,1.002405,3,126,2,100
4,5,7,1.001422,1.002301,1.00137,1.002405,3,126,2,100
5,5,11,1.001422,1.002301,1.00137,1.002405,3,100,2,100
6,5,12,1.001422,1.002301,1.00137,1.002405,3,126,2,100
7,5,14,1.001422,1.002301,1.00137,1.002405,3,126,2,100
8,5,15,1.001422,1.002301,1.00137,1.002405,3,126,2,100
9,5,16,1.001422,1.002301,1.00137,1.002405,3,126,2,100


**Observation**: 
1) There are missing 'seconds_in_bucket". As per Optiver, this means there was no change in those seconds in any of the prices or sizes of the top two bid and ask quotes. We shall see later if these values should be imputed.

2) There are no missing values in any columns.


In [5]:
stock_0_book_train.isna().sum()

time_id              0
seconds_in_bucket    0
bid_price1           0
ask_price1           0
bid_price2           0
ask_price2           0
bid_size1            0
ask_size1            0
bid_size2            0
ask_size2            0
dtype: int64

In [220]:
stock_0_book_train.describe()

,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2
count,917553.000000,917553.000000,917553.000000,917553.000000,917553.000000,917553.000000,917553.000000,917553.000000,917553.000000,917553.000000
mean,15980.056908,293.692015,0.999904,1.000137,0.999864,1.000177,113.717102,100.825350,86.770241,83.203067
std,9381.778917,173.596440,0.003668,0.003696,0.003702,0.003743,108.657209,109.063892,90.602588,94.968388
min,5.000000,0.000000,0.938241,0.944337,0.937213,0.944456,1.000000,1.000000,1.000000,1.000000
25%,7759.000000,142.000000,0.998368,0.999223,0.998184,0.999390,22.000000,14.000000,18.000000,14.000000
50%,15772.000000,292.000000,0.999632,1.000331,0.999472,1.000496,100.000000,93.000000,100.000000,90.000000
75%,23834.000000,444.000000,1.000753,1.001560,1.000587,1.001745,157.000000,117.000000,102.000000,102.000000
max,32767.000000,599.000000,1.045641,1.056892,1.043756,1.057676,3221.000000,16608.000000,4391.000000,16608.000000


In [7]:
stock_0_time_stamps = pd.DataFrame(stock_0_book_train.groupby('time_id')['seconds_in_bucket'].count()).reset_index()
stock_0_time_stamps.head()


,time_id,seconds_in_bucket
0,5,302
1,11,200
2,16,188
3,31,120
4,62,176


**Observation**: The no. of 'seconds_in_buckets' for which data is available varies widely by time_id.

In [8]:
stock_0_time_stamps.describe()

,time_id,seconds_in_bucket
count,3830.000000,3830.000000
mean,16038.933681,239.569974
std,9366.261949,72.150094
min,5.000000,73.000000
25%,7856.500000,188.000000
50%,15852.500000,229.000000
75%,23988.500000,280.000000
max,32767.000000,549.000000


In [252]:
## How many unique time_ids are present in the book training file? And what's the average time_stamps in each bucket ?

time_buckets = stock_0_time_stamps['time_id'].nunique()
avg_time_stamps = stock_0_time_stamps['seconds_in_bucket'].mean()

print(f'There are {time_buckets} unique time buckets')
print(f'Average time stamps per time bucket: {avg_time_stamps}')


There are 3830 unique time buckets
Average time stamps per time bucket: 239.5699738903394


Let's explore the book files of a couple of more stocks to see if the observations are common.


### Stock_id = 50

In [9]:
stock_50_book_train = pd.read_parquet('../Raw_data/book_train.parquet/stock_id=50')
stock_50_book_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2148692 entries, 0 to 2148691
Data columns (total 10 columns):
 #   Column             Dtype  
---  ------             -----  
 0   time_id            int16  
 1   seconds_in_bucket  int16  
 2   bid_price1         float32
 3   ask_price1         float32
 4   bid_price2         float32
 5   ask_price2         float32
 6   bid_size1          int32  
 7   ask_size1          int32  
 8   bid_size2          int32  
 9   ask_size2          int32  
dtypes: float32(4), int16(2), int32(4)
memory usage: 73.8 MB


In [11]:
stock_50_book_train.head()

,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2
0,5,0,1.000800,1.001105,1.000762,1.001219,110,200,100,82
1,5,1,1.000762,1.001105,1.000724,1.001143,100,250,175,100
2,5,2,1.000610,1.001029,1.000571,1.001067,135,250,100,100
3,5,3,1.000610,1.001029,1.000533,1.001067,211,355,107,100
4,5,4,1.000610,1.001029,1.000571,1.001067,100,456,100,100


In [239]:
stock_50_book_train.isna().sum()

time_id              0
seconds_in_bucket    0
bid_price1           0
ask_price1           0
bid_price2           0
ask_price2           0
bid_size1            0
ask_size1            0
bid_size2            0
ask_size2            0
dtype: int64

In [12]:
stock_50_time_stamps = pd.DataFrame(stock_50_book_train.groupby('time_id')['seconds_in_bucket'].count()).reset_index()
stock_50_time_stamps.head()

,time_id,seconds_in_bucket
0,5,595
1,11,530
2,16,572
3,31,521
4,62,480


In [13]:
stock_50_time_stamps.describe()

,time_id,seconds_in_bucket
count,3830.000000,3830.000000
mean,16038.933681,561.016188
std,9366.261949,40.467570
min,5.000000,364.000000
25%,7856.500000,541.000000
50%,15852.500000,575.000000
75%,23988.500000,592.000000
max,32767.000000,600.000000


In [251]:
## How many unique time_ids are present in the book training file? And what's the average time_stamps in each bucket ?

time_buckets = stock_50_time_stamps['time_id'].nunique()
avg_time_stamps = stock_50_time_stamps['seconds_in_bucket'].mean()

print("For stock_id = 50")
print(f'There are {time_buckets} unique time buckets')
print(f'Average time stamps per time bucket: {avg_time_stamps}')

For stock_id = 50
There are 3830 unique time buckets
Average time stamps per time bucket: 561.0161879895561


Stock_id=50 has on average **561** time stamps per time_id as compared to **240** for stock_id=0.


### Stock_id = 100

In [244]:
stock_100_book_train = pd.read_parquet('../Raw_data/book_train.parquet/stock_id=100')
stock_100_book_train.head()


,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2
0,5,0,0.999077,1.000103,0.999009,1.000239,100,7,64,100
1,5,4,0.999214,1.000103,0.999145,1.000239,300,7,310,100
2,5,5,0.999282,1.000239,0.999214,1.000376,700,201,400,36
3,5,6,0.999487,1.000786,0.999419,1.000855,100,101,10,50
4,5,7,0.999624,1.000786,0.999556,1.000855,200,101,100,50


In [245]:
stock_100_book_train.isna().sum()

time_id              0
seconds_in_bucket    0
bid_price1           0
ask_price1           0
bid_price2           0
ask_price2           0
bid_size1            0
ask_size1            0
bid_size2            0
ask_size2            0
dtype: int64

In [253]:
stock_100_time_stamps = pd.DataFrame(stock_100_book_train.groupby('time_id')['seconds_in_bucket'].count()).reset_index()
stock_100_time_stamps.describe()

,time_id,seconds_in_bucket
count,3829.000000,3829.000000
mean,16042.302951,344.448681
std,9365.163672,87.691980
min,5.000000,98.000000
25%,7864.000000,277.000000
50%,15853.000000,343.000000
75%,23994.000000,410.000000
max,32767.000000,578.000000


In [254]:
## How many unique time_ids are present in the book training file? And what's the average time_stamps in each bucket ?

time_buckets = stock_100_time_stamps['time_id'].nunique()
avg_time_stamps = stock_100_time_stamps['seconds_in_bucket'].mean()

print("For stock_id = 100")
print(f'There are {time_buckets} unique time buckets')
print(f'Average time stamps per time bucket: {avg_time_stamps}')


For stock_id = 100
There are 3829 unique time buckets
Average time stamps per time bucket: 344.4486811177853


In [265]:
# Are time_ids common to stocks in the book_train files?

compare_time_stamps = stock_0_time_stamps.merge(stock_50_time_stamps, on='time_id', how='left', suffixes = ['_stk0', '_stk50'])
compare_time_stamps = compare_time_stamps.merge(stock_100_time_stamps, on='time_id', how='left')
compare_time_stamps=compare_time_stamps.rename(columns = {'seconds_in_bucket':'seconds_in_bucket_stk100'})

compare_time_stamps.tail()


,time_id,seconds_in_bucket_stk0,seconds_in_bucket_stk50,seconds_in_bucket_stk100
3825,32751,297,548,469.0
3826,32753,206,556,283.0
3827,32758,188,490,240.0
3828,32763,307,595,407.0
3829,32767,228,534,297.0


In [266]:
compare_time_stamps.isna().sum()

time_id                     0
seconds_in_bucket_stk0      0
seconds_in_bucket_stk50     0
seconds_in_bucket_stk100    1
dtype: int64

In [267]:
compare_time_stamps[compare_time_stamps['seconds_in_bucket_stk100'].isnull()]

,time_id,seconds_in_bucket_stk0,seconds_in_bucket_stk50,seconds_in_bucket_stk100
390,3138,248,598,NaN


On joining the time_stamp files of the 3 stocks, it was found that they all had the same time buckets, except stock_100 which had one missing time bucket (time_id = 3138).


**Key Observations - Book File (Training)**

1. The book file is quite clean - no missing values across 3 files randomly selected.

2. However, the bid and ask data is not present for every second in a time bucket. The no. of seconds for which data is available varies between time buckets (time_id).

3. Data is available for roughly 3830 time buckets(time_id) for each stock. 

4. However, for each stock, the average time stamps per time bucket varies. It is not present for all 600 seconds in each 10-min time bucket. That means there are missing time stamps for each stock within a time bucket.

**Other Important Inputs provided by Optiver**

1. Time_ids are randomly shuffled and do not have a sequential logic i.e. time_id=6 does not follow time_id=5 in actual time.

2. However, time_ids are consistent across stocks i.e. time_id=5 is the same actual time period for all stocks.

3. As per Optiver, the missing time stamps means there was no change in the top two bid/ask prices or sizes, so the missing time stamps can be forward filled using the most recent time stamp data available. **This might be useful to do, especially while analysing volatility of smaller time segments within the 10-min bucket.**

**In summary**: The data for missing 'seconds_in_bucket' to be forward filled in the book_train files.


## 2.0 A peak into the trade data training file

There is a trade_train file available for each of the 112 different stocks. We shall look at this file for a few stocks to understand the data and perform any cleaning, imputing and tidying operations if required.

### Stock_id=0

In [14]:
stock_0_trade_train = pd.read_parquet('../Raw_data/trade_train.parquet/stock_id=0')
stock_0_trade_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123443 entries, 0 to 123442
Data columns (total 5 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   time_id            123443 non-null  int16  
 1   seconds_in_bucket  123443 non-null  int16  
 2   price              123443 non-null  float32
 3   size               123443 non-null  int32  
 4   order_count        123443 non-null  int16  
dtypes: float32(1), int16(3), int32(1)
memory usage: 1.6 MB


In [15]:
stock_0_trade_train.head()

,time_id,seconds_in_bucket,price,size,order_count
0,5,21,1.002301,326,12
1,5,46,1.002778,128,4
2,5,50,1.002818,55,1
3,5,57,1.003155,121,5
4,5,68,1.003646,4,1


In [227]:
stock_0_trade_train.isna().sum()

time_id              0
seconds_in_bucket    0
price                0
size                 0
order_count          0
dtype: int64

In [228]:
stock_0_trade_train.describe()

,time_id,seconds_in_bucket,price,size,order_count
count,123443.000000,123443.000000,123443.000000,123443.000000,123443.000000
mean,15984.113170,297.089296,0.999945,99.685547,3.191449
std,9364.979641,173.408800,0.003425,153.607706,3.348668
min,5.000000,0.000000,0.943890,1.000000,1.000000
25%,7745.000000,146.000000,0.998754,3.000000,1.000000
50%,15824.000000,297.000000,0.999954,33.000000,2.000000
75%,23807.500000,447.000000,1.001141,124.000000,4.000000
max,32767.000000,599.000000,1.045725,5297.000000,79.000000


In [17]:
stock_0_time_stamps_trade = pd.DataFrame(stock_0_trade_train.groupby('time_id')['seconds_in_bucket'].count()).reset_index()
stock_0_time_stamps_trade.head()

,time_id,seconds_in_bucket
0,5,40
1,11,30
2,16,25
3,31,15
4,62,22


In [18]:
stock_0_time_stamps_trade.describe()

,time_id,seconds_in_bucket
count,3830.000000,3830.000000
mean,16038.933681,32.230548
std,9366.261949,14.592698
min,5.000000,2.000000
25%,7856.500000,22.000000
50%,15852.500000,30.000000
75%,23988.500000,40.000000
max,32767.000000,119.000000


In [19]:
## How many unique time_ids are present in the trade training file? And what's the average time_stamps in each bucket ?

time_buckets = stock_0_time_stamps_trade['time_id'].nunique()
avg_time_stamps = stock_0_time_stamps_trade['seconds_in_bucket'].mean()

print("For stock_id = 0")
print(f'There are {time_buckets} unique time buckets in trade file')
print(f'Average time stamps per time bucket: {avg_time_stamps}')


For stock_id = 0
There are 3830 unique time buckets in trade file
Average time stamps per time bucket: 32.230548302872066


Clearly, the trade data is more sparse than book data. stock_id=0 has on average **32** time stamps per time_id in trade file compared to **240** time stamps per time_id in the book file.


### stock_id = 50

In [270]:
stock_50_trade_train = pd.read_parquet('../Raw_data/trade_train.parquet/stock_id=50')
stock_50_trade_train.head()

,time_id,seconds_in_bucket,price,size,order_count
0,5,4,1.000876,10,1
1,5,6,1.000613,102,2
2,5,7,1.000533,1,1
3,5,9,1.000686,200,3
4,5,16,1.000743,237,5


In [271]:
stock_50_trade_train.isna().sum()

time_id              0
seconds_in_bucket    0
price                0
size                 0
order_count          0
dtype: int64

In [272]:
stock_50_trade_train.describe()

,time_id,seconds_in_bucket,price,size,order_count
count,978503.000000,978503.000000,978503.000000,978503.000000,978503.000000
mean,15977.786256,297.907866,0.999983,325.051851,4.959444
std,9394.246601,173.596074,0.003890,893.130994,11.470035
min,5.000000,0.000000,0.946878,1.000000,1.000000
25%,7752.000000,147.000000,0.998333,87.000000,1.000000
50%,15772.000000,297.000000,0.999971,163.000000,3.000000
75%,23836.000000,448.000000,1.001601,350.000000,6.000000
max,32767.000000,599.000000,1.031878,161326.000000,3435.000000


In [274]:
stock_50_time_stamps_trade = pd.DataFrame(stock_50_trade_train.groupby('time_id')['seconds_in_bucket'].count()).reset_index()
stock_50_time_stamps_trade.describe()

,time_id,seconds_in_bucket
count,3830.000000,3830.000000
mean,16038.933681,255.483812
std,9366.261949,90.391497
min,5.000000,74.000000
25%,7856.500000,184.250000
50%,15852.500000,246.000000
75%,23988.500000,315.000000
max,32767.000000,566.000000


In [276]:
## How many unique time_ids are present in the trade training file? And what's the average time_stamps in each bucket ?

time_buckets = stock_50_time_stamps_trade['time_id'].nunique()
avg_time_stamps = stock_50_time_stamps_trade['seconds_in_bucket'].mean()

print("For stock_id = 50")
print(f'There are {time_buckets} unique time buckets')
print(f'Average time stamps per time bucket: {avg_time_stamps}')

For stock_id = 50
There are 3830 unique time buckets
Average time stamps per time bucket: 255.48381201044387


stock_id=50 is a **more actively traded stock** as compared to stock_id=0 with **8x** the no. of trades per time_id. Still, the trade time_stamps are much lower than book time_stamps (561).


### Stock_id=100

In [278]:
stock_100_trade_train = pd.read_parquet('../Raw_data/trade_train.parquet/stock_id=100')
stock_100_trade_train.head()

,time_id,seconds_in_bucket,price,size,order_count
0,5,5,1.000103,7,2
1,5,6,1.000239,201,4
2,5,9,1.000444,1,1
3,5,28,1.000376,12,3
4,5,30,1.000444,3,1


In [279]:
stock_100_trade_train.isna().sum()

time_id              0
seconds_in_bucket    0
price                0
size                 0
order_count          0
dtype: int64

In [280]:
stock_100_trade_train.describe()

,time_id,seconds_in_bucket,price,size,order_count
count,293993.000000,293993.000000,293993.000000,293993.000000,293993.000000
mean,15958.674499,297.300997,0.999931,219.931767,3.657485
std,9379.786674,173.395555,0.006088,538.670696,4.524530
min,5.000000,0.000000,0.921751,1.000000,1.000000
25%,7716.000000,147.000000,0.997797,32.000000,1.000000
50%,15894.000000,296.000000,0.999931,105.000000,2.000000
75%,23751.000000,448.000000,1.001932,268.000000,5.000000
max,32767.000000,599.000000,1.069981,133273.000000,574.000000


In [281]:
stock_100_time_stamps_trade = pd.DataFrame(stock_100_trade_train.groupby('time_id')['seconds_in_bucket'].count()).reset_index()
stock_100_time_stamps_trade.describe()

,time_id,seconds_in_bucket
count,3829.000000,3829.000000
mean,16042.302951,76.780622
std,9365.163672,38.022239
min,5.000000,10.000000
25%,7864.000000,50.000000
50%,15853.000000,68.000000
75%,23994.000000,94.000000
max,32767.000000,278.000000


In [282]:

time_buckets = stock_100_time_stamps_trade['time_id'].nunique()
avg_time_stamps = stock_100_time_stamps_trade['seconds_in_bucket'].mean()

print("For stock_id = 100")
print(f'There are {time_buckets} unique time buckets')
print(f'Average time stamps per time bucket: {avg_time_stamps}')

For stock_id = 100
There are 3829 unique time buckets
Average time stamps per time bucket: 76.78062157221207


As with book data, stock_id=100 has trade data for 1 less time bucket (3829 vs 3830 for other stocks). It's possible that it is missing it for the same time_id (3138). Let's check that. 

In [284]:
compare_time_stamps = stock_0_time_stamps_trade.merge(stock_50_time_stamps_trade, on='time_id', how='left', suffixes = ['_stk0', '_stk50'])
compare_time_stamps = compare_time_stamps.merge(stock_100_time_stamps_trade, on='time_id', how='left')
compare_time_stamps=compare_time_stamps.rename(columns = {'seconds_in_bucket':'seconds_in_bucket_stk100'})

compare_time_stamps.tail()



,time_id,seconds_in_bucket_stk0,seconds_in_bucket_stk50,seconds_in_bucket_stk100
3825,32751,52,187,67.0
3826,32753,28,161,63.0
3827,32758,36,142,40.0
3828,32763,53,378,110.0
3829,32767,29,141,88.0


In [285]:
compare_time_stamps[compare_time_stamps['seconds_in_bucket_stk100'].isnull()]


,time_id,seconds_in_bucket_stk0,seconds_in_bucket_stk50,seconds_in_bucket_stk100
390,3138,35,386,NaN


**Key Observations - Trade File (Training)**

1. The trade file is also quite clean - no missing values.
2. As compared to book data, the trade data is sparse. 
3. Data is available for 3830 time buckets(time_id) for most stocks, same as the book file. 
4. Additionally, there is data for the size of shares traded and no. of orders executed in a particular second.
5. No data for certain 'seconds_in_bucket' means no trades happened in those seconds. Here the data cannot be forward filled, as doing that would imply more trades and thus be erroneous

**In summary**: The trade files are clean and no missing values need to be imputed.


### 3.0 Train.csv and test.csv files

This file contains the target for each time_id. The **target** is the realized volatility in the 10-min time window following the time_id in question, so it is the value that needs to be predicted based on book and trade data available for a particular time_id.


In [82]:
train_file = pd.read_csv('../Raw_data/train.csv')
train_file.head()


,stock_id,time_id,target
0,0,5,0.004136
1,0,11,0.001445
2,0,16,0.002168
3,0,31,0.002195
4,0,62,0.001747


In [5]:
train_file.isna().sum()

stock_id    0
time_id     0
target      0
dtype: int64

In [6]:
time_buckets = pd.DataFrame(train_file.groupby('stock_id')['time_id'].count()).reset_index()
time_buckets = time_buckets.rename(columns = {'time_id':'no_of_time_ids_present'})
time_buckets.head()

,stock_id,no_of_time_ids_present
0,0,3830
1,1,3830
2,2,3830
3,3,3830
4,4,3830


In [7]:
time_buckets['no_of_time_ids_present'].unique()

array([3830, 3829, 3815, 3820], dtype=int64)

In [8]:
# Based on above figures, some stocks have data for less than 3830 time buckets. Let's check for how many stocks this happens.

time_buckets_count = pd.DataFrame(time_buckets.groupby('no_of_time_ids_present')['stock_id'].count()).reset_index()
time_buckets_count = time_buckets_count.rename(columns = {'stock_id':'no_of_stocks'})
time_buckets_count

,no_of_time_ids_present,no_of_stocks
0,3815,1
1,3820,1
2,3829,3
3,3830,107


**Observation**: So all except 5 stocks, all other stocks have target data present for **3830 time ids**  

In [296]:
# Now let's look at the test file

test_file = pd.read_csv('../Raw_data/test.csv')
test_file.head()


,stock_id,time_id,row_id
0,0,4,0-4
1,0,32,0-32
2,0,34,0-34


In [297]:
test_file.shape

(3, 3)

The test file just has dummy data. As per Optiver, the sample data in test file is meant to give a feeling about the data structure, once the code notebook is committed, the code will be running against the real test set in the background and leaderboard will be updated.



## 4.0 Imputing Missing Values & Splitting Training data into Training and Test sets

As there is no access to test data, it needs to be carved out from the training data.
There is data for approx. 3830 time buckets for most stocks. As this is not a very large no., we shall do a 80-20 split. 

The training/test data must be split for each stock for the same time buckets. This is so that potential models can analyse volatility across stocks for the same time buckets and learn if this market-wide volatility has any bearing in predicting future volatility of individual stocks.

To do this, the following steps shall be followed:

**Step 1**: Do a 80-20 random split of the 3830 time_id values.

**Step 2**: For each of the 112 stocks, split the following 3 files using the above split of the time_ids:
            a) book_train.parquet
            b) trade_train.parquet
            c) train.csv

**Step 3**: Save the resulting dataframes as csv files for each stock.


In [83]:
time_ids = train_file['time_id'].unique()
time_ids

array([    5,    11,    16, ..., 32758, 32763, 32767], dtype=int64)

In [84]:
len(time_ids)

3830

In [85]:
train_time_ids, test_time_ids = train_test_split(time_ids, test_size = 0.2, random_state = 123)
train_time_ids.shape, test_time_ids.shape

((3064,), (766,))

In [86]:
np.min(train_time_ids), np.max(train_time_ids)

(5, 32767)

In [87]:
np.min(test_time_ids), np.max(test_time_ids)

(147, 32704)

In [88]:

def df_split(df, train_ids, test_ids, split_column):
    # function to split the book, trade and train files
    
    df_train = df[df[split_column].isin(train_ids)]
    df_test = df[df[split_column].isin(test_ids)]
    
    return df_train, df_test


In [28]:

target_tr, target_te = df_split(train_file, train_time_ids, test_time_ids, 'time_id')

target_tr.shape, target_te.shape


((343143, 3), (85789, 3))

In [31]:
datapath = "../WIP_data"

save_file(target_tr, 'target_training_data.csv', datapath)
save_file(target_te, 'target_test_data.csv', datapath)


Directory ../WIP_data was created.
Writing file.  "../WIP_data\target_training_data.csv"
Writing file.  "../WIP_data\target_test_data.csv"


In [107]:
file_list = glob.glob('../Raw_data/book_train.parquet/stock_id=???')
datapath_train = "../WIP_data/book/train"
datapath_test = "../WIP_data/book/test"

list_seconds = np.arange(0,600)

for file_name in file_list:
    stock_id = file_name.split("=")[1]
    df_book = pd.read_parquet(file_name)
    df_book['stock_id'] = stock_id
    df_new = pd.DataFrame()
    for time_id in df_book['time_id'].unique():
        df_temp = df_book[df_book['time_id'] == time_id].set_index('seconds_in_bucket')
        df_temp = df_temp.reindex(list_seconds).reset_index()
        df_temp = df_temp.ffill()
        df_new = pd.concat([df_new, df_temp], ignore_index=True)
    
    df_train, df_test = df_split(df_new, train_time_ids, test_time_ids, 'time_id')
    filename_train = "stock_"+str(stock_id)+"_book_train.csv"
    filename_test = "stock_"+str(stock_id)+"_book_test.csv"
    save_file(df_train, filename_train, datapath_train)
    save_file(df_test, filename_test, datapath_test)
    
    

Writing file.  "../WIP_data/book/train\stock_100_book_train.csv"
Writing file.  "../WIP_data/book/test\stock_100_book_test.csv"
Writing file.  "../WIP_data/book/train\stock_101_book_train.csv"
Writing file.  "../WIP_data/book/test\stock_101_book_test.csv"
Writing file.  "../WIP_data/book/train\stock_102_book_train.csv"
Writing file.  "../WIP_data/book/test\stock_102_book_test.csv"
Writing file.  "../WIP_data/book/train\stock_103_book_train.csv"
Writing file.  "../WIP_data/book/test\stock_103_book_test.csv"
Writing file.  "../WIP_data/book/train\stock_104_book_train.csv"
Writing file.  "../WIP_data/book/test\stock_104_book_test.csv"
Writing file.  "../WIP_data/book/train\stock_105_book_train.csv"
Writing file.  "../WIP_data/book/test\stock_105_book_test.csv"
Writing file.  "../WIP_data/book/train\stock_107_book_train.csv"
Writing file.  "../WIP_data/book/test\stock_107_book_test.csv"
Writing file.  "../WIP_data/book/train\stock_108_book_train.csv"
Writing file.  "../WIP_data/book/test\s

In [47]:
import pyarrow as pa
import pyarrow.parquet as pq

file_list = glob.glob('../Raw_data/trade_train.parquet/stock_id=*')
datapath_train = "../WIP_data/trade/train"
datapath_test = "../WIP_data/trade/test"

for file_name in file_list:
    stock_id = file_name.split("=")[1]
    df_book = pd.read_parquet(file_name)
    df_book['stock_id'] = stock_id
    df_train, df_test = df_split(df_book, train_time_ids, test_time_ids, 'time_id')
    filename_train = "stock_"+str(stock_id)+"_trade_train.csv"
    filename_test = "stock_"+str(stock_id)+"_trade_test.csv"
    save_file(df_train, filename_train, datapath_train)
    save_file(df_test, filename_test, datapath_test)
    

Directory ../WIP_data/trade/train was created.
Writing file.  "../WIP_data/trade/train\stock_0_trade_train.csv"
Directory ../WIP_data/trade/test was created.
Writing file.  "../WIP_data/trade/test\stock_0_trade_test.csv"
Writing file.  "../WIP_data/trade/train\stock_1_trade_train.csv"
Writing file.  "../WIP_data/trade/test\stock_1_trade_test.csv"
Writing file.  "../WIP_data/trade/train\stock_10_trade_train.csv"
Writing file.  "../WIP_data/trade/test\stock_10_trade_test.csv"
Writing file.  "../WIP_data/trade/train\stock_100_trade_train.csv"
Writing file.  "../WIP_data/trade/test\stock_100_trade_test.csv"
Writing file.  "../WIP_data/trade/train\stock_101_trade_train.csv"
Writing file.  "../WIP_data/trade/test\stock_101_trade_test.csv"
Writing file.  "../WIP_data/trade/train\stock_102_trade_train.csv"
Writing file.  "../WIP_data/trade/test\stock_102_trade_test.csv"
Writing file.  "../WIP_data/trade/train\stock_103_trade_train.csv"
Writing file.  "../WIP_data/trade/test\stock_103_trade_tes

Writing file.  "../WIP_data/trade/test\stock_47_trade_test.csv"
Writing file.  "../WIP_data/trade/train\stock_48_trade_train.csv"
Writing file.  "../WIP_data/trade/test\stock_48_trade_test.csv"
Writing file.  "../WIP_data/trade/train\stock_5_trade_train.csv"
Writing file.  "../WIP_data/trade/test\stock_5_trade_test.csv"
Writing file.  "../WIP_data/trade/train\stock_50_trade_train.csv"
Writing file.  "../WIP_data/trade/test\stock_50_trade_test.csv"
Writing file.  "../WIP_data/trade/train\stock_51_trade_train.csv"
Writing file.  "../WIP_data/trade/test\stock_51_trade_test.csv"
Writing file.  "../WIP_data/trade/train\stock_52_trade_train.csv"
Writing file.  "../WIP_data/trade/test\stock_52_trade_test.csv"
Writing file.  "../WIP_data/trade/train\stock_53_trade_train.csv"
Writing file.  "../WIP_data/trade/test\stock_53_trade_test.csv"
Writing file.  "../WIP_data/trade/train\stock_55_trade_train.csv"
Writing file.  "../WIP_data/trade/test\stock_55_trade_test.csv"
Writing file.  "../WIP_data/

In [2]:
s0_book_tr = pd.read_csv('../WIP_data/book/train/stock_0_book_train.csv')
s0_book_tr.head(20)


,seconds_in_bucket,time_id,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2,stock_id
0,0,5.0,1.001422,1.002301,1.00137,1.002353,3.0,226.0,2.0,100.0,0
1,1,5.0,1.001422,1.002301,1.00137,1.002353,3.0,100.0,2.0,100.0,0
2,2,5.0,1.001422,1.002301,1.00137,1.002353,3.0,100.0,2.0,100.0,0
3,3,5.0,1.001422,1.002301,1.00137,1.002353,3.0,100.0,2.0,100.0,0
4,4,5.0,1.001422,1.002301,1.00137,1.002353,3.0,100.0,2.0,100.0,0
5,5,5.0,1.001422,1.002301,1.00137,1.002405,3.0,100.0,2.0,100.0,0
6,6,5.0,1.001422,1.002301,1.00137,1.002405,3.0,126.0,2.0,100.0,0
7,7,5.0,1.001422,1.002301,1.00137,1.002405,3.0,126.0,2.0,100.0,0
8,8,5.0,1.001422,1.002301,1.00137,1.002405,3.0,126.0,2.0,100.0,0
9,9,5.0,1.001422,1.002301,1.00137,1.002405,3.0,126.0,2.0,100.0,0


In [3]:
s0_book_tr.shape

(1838400, 11)

# Data Wrangling: Conclusion 


**Key Observations - Book File (Training)**

1. The book file is quite clean - no missing values across 3 files randomly selected.

2. However, the bid and ask data is not present for every second in a time bucket. The no. of seconds for which data is available varies between time buckets (time_id).

3. Data is available for roughly 3830 time buckets(time_id) for each stock. 

4. However, for each stock, the average time stamps per time bucket varies. It is not present for all 600 seconds in each 10-min time bucket. That means there are missing time stamps for each stock within a time bucket.

**Key Observations - Trade File (Training)**

1. The trade file is also quite clean - no missing values.
2. As compared to book data, the trade data is sparse. 
3. Data is available for 3830 time buckets(time_id) for most stocks, same as the book file. 
4. Additionally, there is data for the size of shares traded and no. of orders executed in a particular second.

**Key Observations - Train.csv and test.csv**
1. The train file has the target to be predicted for each time_id. 
2. Most stocks have target data for 3830 time_ids barring 5 stocks that have slightly fewer time buckets.
3. There is no data in test file. As per Optiver, once the code notebooks ae committed on kaggle, they would run them with test data in the background and update leaderboard.

**Other Important Inputs provided by Optiver**

1. Time_ids are randomly shuffled and do not have a sequential logic i.e. time_id=6 does not follow time_id=5 in actual time.

2. However, time_ids are consistent across stocks i.e. time_id=5 is the same actual time period for all stocks.

3. As per Optiver, the missing time stamps means there was no change in the top two bid/ask prices or sizes, so the missing time stamps can be forward filled using the most recent time stamp data available. 

**In Summary**: 
1) In the book files, the missing 'seconds_in_bucket' observations were imputed using forward-fill as recommended by Optiver.

2) The training files were split in training and test sets with a 80:20 ratio.